In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [12]:
from Korpora import Korpora
Corpus = Korpora.load("korean_petitions")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/petitions_archive
    References :

    청와대 국민청원 게시판의 데이터를 월별로 수집한 것입니다.
    청원은 게시판에 글을 올린 뒤, 한달 간 청원이 진행됩니다.
    수집되는 데이터는 청원종료가 된 이후의 데이터이며, 청원 내 댓글은 수집되지 않습니다.
    단 청원의 동의 개수는 수집됩니다.
    자세한 내용은 위의 repository를 참고하세요.

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `korean_petitions` is already installed at C:\Users\ImedisynRnD2\Korpora\korean_petitions\petitions_2017-08
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\ImedisynRnD2\Korpora\korean_petitions\petitions_2017-09
[Korpora] Corpus `korean_petitions` is already

In [3]:
import kss
import re
from konlpy.tag import Mecab

class Prep():
        
    def Clean_text(self, text): # noisy한 문장을 cleaning
        self.sentences = []
        
        for i in range(len(text)):
            if (i+1) % 10 == 0:
                print(str(i+1) + 'th sentence is spliting...')
        
            temp = re.sub("[^가-힣a-z0-9.?]", ' ', text[i]) #한글, 영어, 숫자, 온점, 물음표가 아닌 것을 공백으로 삭제
            temp = re.sub("[.]{2,}", ".", temp) # 반복되는 온점 (...) 삭제
            temp = re.sub("[?]{2,}", "?", temp) # 반복되는 물음표 (?) 삭제
            temp = re.sub("[!]{2,}", "!", temp) # 반복되는 느낌표 (!) 삭제
            temp = re.sub("[' ']{2,}", " ", temp) # 반복되는 공백 삭제 
            temp = kss.split_sentences(temp)  #문장 분리

            for tmp in temp:
                self.sentences.append(tmp)
        
        return self.sentences

        
    def Tokenizer(self, sentences): # cleaned data에 대한 형태소 분석 후 vocabulary 구축
        
        self.corpus = []
        self.words = []
        self.vocab = []
        
        tokenizer = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")
        
        for sent in self.sentences:
            temp = tokenizer.morphs(sent)
            self.corpus.append(temp)
            
            for tmp in temp:
                self.words.append(tmp)

        self.vocab = set(self.words)
        
        return self.corpus, self.words, self.vocab
    
    def Make_dict(self,vocab): # 만들어진 Vocabulary를 기준으로 단어-정수 및 정수-단어 인덱싱 dictionary 생성
        self.word_dict = {w: i for i, w in enumerate(vocab)}
        self.index_dict = {i: w for i, w in enumerate(vocab)}
        return self.word_dict, self.index_dict 
    
    def Get_clean(self, text): #위의 세가지 메소드를 한번에 진행                
        sentences =  self.Clean_text(text)
        corpus, words, vocab = self.Tokenizer(sentences)
        word_dict, index_dict = self.Make_dict(vocab)
        
        return sentences, corpus, words, vocab, word_dict, index_dict

In [239]:
class Make_pairs(): # Input과 target, label을 만들어주는 클래스
    
    def __init__(self):
        
        from collections import defaultdict
        self.inputs = [] 
        self.targets = []
        self.labels = []
        self.vocab_size = len(vocab)
        
        # Negative sampling 정의
        self.wordFreq = defaultdict(int)
        for word in words:
            self.wordFreq[word] += 1
        
        self.SumFreq = sum([freq**(3/4) for freq in self.wordFreq.values()]) # 분모
        self.wordProb = {word:((freq**(3/4))/self.SumFreq) for word, freq in self.wordFreq.items()} #샘플링 확률분포
        
    def Draw(self,n_sample): # n_sample : sample을 몇 개를 뽑을건지?
        negsam = []
        for i in range(n_sample):
            negsam.append(np.random.choice(list(self.wordProb.keys()), p=list(self.wordProb.values()))) #확률분포 P에 따른 추출
        
        return negsam
    
    def Get_pairs(self, window_size, n_negsam):
        for c in corpus: # for each sentence          
            for idx in range(len(c)): # for each idx
                start = max(0,idx - window_size)
                tail = min(idx + window_size, len(c)-1)
            
                #add negative samples to context
                context = c[start:idx] + c[idx+1:tail+1]
                needed_negsam = 2*window_size+n_negsam - len(context) # needed number of negative samples
                negsam = self.Draw(needed_negsam) # draw negative samples
            
                #stack pairs        
                
                in_onehot = torch.zeros(self.vocab_size) #make input one-hot
                in_onehot[word_dict[c[idx]]] = 1.0
                
                out = context+negsam
                out_onehot = torch.zeros(len(out), self.vocab_size) #make output one-hot

                for idx, txt in enumerate(out):
                    out_onehot[idx,word_dict[txt]] = 1.0
                
                self.inputs.append(in_onehot)
                self.targets.append(out_onehot)
                self.labels.append([1]*len(context) + [0]*needed_negsam)
                   
        return self.inputs, self.targets, self.labels   

In [250]:
## from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.inputs = inputs
        self.targets = targets
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        inputs = self.inputs[idx]
        targets = self.targets[idx]
        labels = self.labels[idx]
                            
        return inputs, targets, labels

In [286]:
from torch.autograd import Variable
import torch.optim as optim
def train(inputs, targets, labels, emb_dim, lr, n_epoch, batch):
    
    vocab_size = len(vocab)
    target_size = len(targets[0])
    center_mat = Variable(torch.nn.init.xavier_normal_(torch.empty(emb_dim, vocab_size)).cuda(),requires_grad=True) # W mat
    context_mat = Variable(torch.nn.init.xavier_normal_(torch.empty(vocab_size, emb_dim)).cuda(),requires_grad=True) # W' mat
    
    dataset = CustomDataset()
    dataloader = DataLoader(dataset, batch_size=batch, shuffle=True)
    
    for epoch in range(n_epoch): 
        loss_value = 0
        
        
        for batch_idx, samples in enumerate(dataloader):
            I,T,L = samples
        
            x = Variable(I.cuda()) # word index of input
            target = Variable(T.cuda()) # word index of target
            h = torch.matmul(center_mat,x.T) # look-up table

            tar = torch.matmul(target,context_mat)
            z = torch.matmul(tar,h.T.unsqueeze(2))             
            y = torch.LongTensor(L).cuda()

            y_hat = F.log_softmax(z, dim=0).squeeze()
            
            loss = 0
            for i in range(batch):
                loss_tmp = F.nll_loss(y_hat[i], y[i])
                loss+=loss_tmp
            
            loss_value += loss.item()
            loss.backward()

            center_mat.data -= lr * center_mat.grad.data # update center_mat
            context_mat.data -= lr * context_mat.grad.data # update context_mat with context_tmp

            # initialize gradient after update
            center_mat.grad.data.zero_() 
            context_mat.grad.data.zero_()

        if epoch % 100 == 0:
            print(f"Loss at this epoch {epoch+100}: {loss_value / vocab_size}")
        
    return center_mat         

In [281]:
vocab_size = len(vocab)
target_size = len(targets[0])
center_mat = Variable(torch.nn.init.xavier_normal_(torch.empty(emb_dim, vocab_size)).cuda(),requires_grad=True) # W mat
context_mat = Variable(torch.nn.init.xavier_normal_(torch.empty(vocab_size, emb_dim)).cuda(),requires_grad=True) # W' mat

dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=20, shuffle=True)

epoch = 0
for batch_idx, samples in enumerate(dataloader):
    I,T,L = samples
    if batch_idx == 0:
        x = Variable(I.cuda()) # word index of input
        target = Variable(T.cuda()) # word index of target
        h = torch.matmul(center_mat,x.T) # look-up table

        tar = torch.matmul(target,context_mat)
        z = torch.matmul(tar,h.T.unsqueeze(2))             
        y = torch.LongTensor(L).cuda()

        y_hat = F.log_softmax(z, dim=0).squeeze()
        
        
        print(np.shape(tar))
        print(np.shape(h))
        print(np.shape(z))
        print(np.shape(y_hat))
        print(np.shape(y))
        
        loss = F.nll_loss(y_hat, y)

torch.Size([20, 26, 30])
torch.Size([30, 20])
torch.Size([20, 26, 1])
torch.Size([20, 26])
torch.Size([20, 26])
tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [9]:
# from torch.autograd import Variable
# import torch.optim as optim
# def train(inputs, targets, labels, emb_dim, lr, n_epoch):
    
#     vocab_size = len(vocab)
#     target_size = len(targets[0])
#     center_mat = Variable(torch.nn.init.xavier_normal_(torch.empty(emb_dim, vocab_size)).cuda(),requires_grad=True) # W mat
#     context_mat = Variable(torch.nn.init.xavier_normal_(torch.empty(vocab_size, emb_dim)).cuda(),requires_grad=True) # W' mat
    
#     for epoch in range(n_epoch): 
#         loss_value = 0
#         for batch in range(len(inputs)):
            
#             x = Variable(inputs[batch].cuda()) # word index of input
#             target = Variable(targets[batch].cuda()) # word index of target
#             h = torch.matmul(center_mat,x) # look-up table
            
#             tar = torch.matmul(target,context_mat)
#             z = torch.matmul(tar,h)             
#             y = torch.LongTensor(labels[batch]).cuda()

#             y_hat = F.log_softmax(z, dim=0)
#             loss = F.nll_loss(y_hat, y)
#             loss_value += loss.item()
#             loss.backward()
            
#             center_mat.data -= lr * center_mat.grad.data # update center_mat
#             context_mat.data -= lr * context_mat.grad.data # update context_mat with context_tmp
            
#             # initialize gradient after update
#             center_mat.grad.data.zero_() 
#             context_mat.grad.data.zero_()
        
#         if epoch % 100 == 0:
#             print(f"Loss at this epoch {epoch+100}: {loss_value / vocab_size}")
        
#     return center_mat         

In [252]:
def TSNEplot(wordvec, vocab, perplexity):
    
    #matplotlib 패키지 한글 깨짐 처리
    import matplotlib.pyplot as plt
    %matplotlib notebook
    plt.rc('font', family='Malgun Gothic') #윈도우, 구글 콜랩
    plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결

    from sklearn.manifold import TSNE
    
    wordvec_2d = TSNE(n_components=2, perplexity=perplexity, learning_rate='auto', init='random').fit_transform(wordvec)
    plt.scatter(wordvec_2d[:,0], wordvec_2d[:,1], s=0.5)

    test_words=[]
    for i in range(len(vocab)):
        test_words.append(index_dict[i])
    
    for i, word in enumerate(test_words):
        plt.annotate(word, xy=(wordvec_2d[i, 0], wordvec_2d[i, 1]))
    
    return wordvec_2d    

In [287]:
# Library import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


all_text = Corpus.train.texts

text = []
for i in range(10):              # 게시글 10개만 추출
    text.append(all_text[i])

#parameters
emb_dim = 30
window_size = 3
n_negsam = 20
lr = 0.01
n_epoch = 500

# sentences, corpus, words, vocab, word_dict, index_dict = Prep().Get_clean(text) # 전처리
# inputs, targets, labels = Make_pairs().Get_pairs(window_size, n_negsam) # Input / target / label 생성
center_mat = train(inputs, targets, labels, emb_dim, lr, n_epoch, batch=20) # Training

# lembedded = center_mat.to('cpu').detach().numpy()
# wordvec_2d = TSNEplot(embedded.T, vocab, perplexity=4) # TNSE plotting

IndexError: index 10 is out of bounds for dimension 0 with size 10